In [46]:
DL = '/Users/damiansp/Downloads'

In [39]:
!rm -rf $DL/*

In [40]:
from datetime import datetime, timedelta
import os

import numpy as np
import pandas as pd
import xlrd

In [71]:
SYMBOLS = [
    'AAPL', 'ABMD', 'ABNB', 'ACN', 'ADBE', 'ADYEY', 'AMD', 'AMGN',
    'AMZN', 'ANET', 'ASML', 'AXON', 'BROS', 'BYND', 'CHWY', 'CRWD',
    'DDOG', 'DOCS', 'DOCU', 'DUOL', 'EDIT', 'ETSY', 'FLGT', 'GH',
    'GMED', 'GOOG', 'IDXX', 'INTG', 'ISRG', 'JD', 'LRCX', 'LSPD',
    'LULU', 'MA', 'MDB', 'MELI', 'MNST', 'MSFT', 'MTCH', 'NET', 'NVCR',
    'NVDA', 'ODFL', 'OKTA', 'OPEN', 'PAC', 'PANW', 'PAYC', 'PINS',
    'PTON', 'PYPL', 'QDEL', 'RBLX', 'RDFN', 'RGEN', 'RIVN', 'RMD',
    'ROKU', 'RVLV', 'SBUX', 'SE', 'SHOP', 'SNOW', 'TASK', 'TDOC',
    'TEAM', 'TREX', 'TSCO', 'TSLA', 'TTD', 'TTWO', 'TWLO', 'TWTR',
    'TYL', 'UPST', 'VEEV', 'VLD', 'VRTX', 'XPEV', 'ZEN', 'ZM']
LINGERERS = [
    'AGI', 'AMRK', 'APT', 'BTTR', 'CENN', 'CTGO', 'CVE', 'DTST', 'EP',
    'FCUV', 'FKWL', 'FSI', 'GCBC', 'GGE', 'GME', 'HBM', 'INTZ', 'KREF',
    'LFMD', 'MCVT', 'MIY', 'MNMD', 'MOBQ', 'PACB', 'PETV', 'POWW',
    'PRPH', 'RFP', 'RVYL', 'SOBR', 'VIRT', 'WAVD', 'WKHS', 'ZEST',
    'ZYXI']
BEST_WEIGHTED = [
    'AMEH', 'BYRN', 'CEF', 'CGAU', 'CIZN', 'DYAI', 'FNV', 'HMY',
    'HNRG', 'IDR', 'IEI', 'MICS', 'MMU', 'MNP', 'MTA', 'MUE', 'NAN',
    'NPV', 'OMQS', 'RVP']
SYMBOLS = sorted(list(set(SYMBOLS + LINGERERS + BEST_WEIGHTED)))

In [42]:
!ls $DL

In [43]:
#!head -5 $DL/Positions.csv

In [72]:
def check_symbols(df):
    all_present = True
    for symbol in df.index.unique():
        if symbol not in SYMBOLS:
            print(f'{symbol} not in SYMBOLS')
            all_present = False
    if all_present:
        print('All clear')

In [73]:
def convert_value(s):
    return round(float(s.replace('$', '').replace(',', '')))

In [74]:
binder = pd.DataFrame(index=SYMBOLS)
binder.head()

""
AAPL
ABMD
ABNB
ACN
ADBE


### E*Trade

In [75]:
TODAY = datetime.now().date()
TODAY
DONGMEI = False

In [76]:
df = pd.read_csv(
    f'{DL}/Positions.csv', index_col=0, skiprows=1
)[['Market Value']].rename(columns={'Market Value': 'ET'})
df.index = map(lambda x: x.split()[0], df.index)
df.drop(index=['FNRG', 'Portfolio', 'Cash'], inplace=True)
df.head()

,ET
AAPL,2874.5099999999998
ABMD,2263.7400000000002
ABNB,195.54
ACN,286.5
ADYEY,3218.5


In [77]:
if DONGMEI:
    et_df = (
        pd.read_csv(
            f'{DL}/2022-04-29.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['Portfolio', 'Cash'], inplace=True)
else:
    et_df = (
        pd.read_csv(
            f'{DL}/Positions.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['FNRG', 'Portfolio', 'Cash'], inplace=True)
    et_df.ET = et_df.ET.apply(
        lambda x: round(float(x)) if x != '--' else x)
et_df.head()

,ET
AAPL,2875
ABMD,2264
ABNB,196
ACN,286
ADYEY,3218


In [78]:
#et_df.drop(index=['98986CONT'], inplace=True)
et_df.head()

,ET
AAPL,2875
ABMD,2264
ABNB,196
ACN,286
ADYEY,3218


In [79]:
et_df.loc[et_df.ET == '--']
#et_df.loc['90184L102', 'ET'] = 53.7

,ET


In [80]:
check_symbols(et_df)

All clear


In [81]:
#et_df = et_df[1:]

In [82]:
out = pd.concat([binder, et_df], axis=1).fillna(0)
out.ET = out.ET.astype(float).apply(lambda x: int(round(x)))
out.head()

,ET
AAPL,2875
ABMD,2264
ABNB,196
ACN,286
ADBE,0


### Fidelity

In [83]:
today = datetime.strftime(TODAY, '%b-%d-%Y')
filename = f'Portfolio_Positions_{today}.csv'
filename

'Portfolio_Positions_Nov-18-2022.csv'

In [84]:
!ls $DL

Portfolio_Positions_Nov-18-2022.csv
Positions.csv
Positions_damiansp_Stocks_2022_11_18.xls
clean.csv


In [85]:
#df_raw = pd.read_csv(f'{DL}/{filename.replace("-24-", "-25-")}')
df_raw = pd.read_csv(f'{DL}/{filename}')
df_raw.head()

,Account Number,Account Name,Symbol,Description,Quantity,Last Price,Last Price Change,Current Value,Today's Gain/Loss Dollar,Today's Gain/Loss Percent,Total Gain/Loss Dollar,Total Gain/Loss Percent,Percent Of Account,Cost Basis,Cost Basis Per Share,Type
0,218520300,Rollover IRA,SPAXX**,FIDELITY GOVERNMENT MONEY MARKET,173.77,$1.00,$0.00,$173.77,NaN,NaN,NaN,NaN,0.50%,NaN,NaN,Cash
1,218520300,Rollover IRA,ABMD,ABIOMED INC COM USD0.01,4.00,$377.29,+$0.43,$1509.16,+$1.72,+0.11%,+$490.47,+48.14%,4.34%,$1018.69,$254.67,Cash
2,218520300,Rollover IRA,ADYEY,ADYEN NV UNSPON ADS EACH REP 0.01 ORD SHS,13.00,$15.70,+$0.416,$204.10,+$5.40,+2.72%,-$10.59,-4.94%,0.59%,$214.69,$16.51,Cash
3,218520300,Rollover IRA,AMD,ADVANCED MICRO DEVICES INC,18.00,$73.57,-$0.33,$1324.26,-$5.94,-0.45%,-$22.65,-1.69%,3.81%,$1346.91,$74.83,Cash
4,218520300,Rollover IRA,BROS,DUTCH BROS INC CL A,22.00,$34.11,+$0.68,$750.42,+$14.96,+2.03%,-$212.07,-22.04%,2.16%,$962.49,$43.75,Cash


In [86]:
df = df_raw[['Account Name', 'Symbol', 'Current Value']].dropna()
df['Current Value'] = df['Current Value'].apply(convert_value)
df['Current Value'] = df['Current Value'].fillna(0).astype(int)
df = df[((df.Symbol != 'SPAXX**') & (df.Symbol != 'Pending Activity'))]
df.head()

,Account Name,Symbol,Current Value
1,Rollover IRA,ABMD,1509
2,Rollover IRA,ADYEY,204
3,Rollover IRA,AMD,1324
4,Rollover IRA,BROS,750
5,Rollover IRA,BYND,54


In [87]:
accounts = {account_name: df[df['Account Name'] == account_name]
            for account_name in df['Account Name'].unique()}
for acct in accounts.keys():
    accounts[acct].index = accounts[acct].Symbol
    accounts[acct] = (
        accounts[acct]
        .rename(columns={'Current Value': acct})
        .drop(columns=['Account Name', 'Symbol']))
    
accounts['Rollover IRA'].head()

,Rollover IRA
Symbol,
ABMD,1509
ADYEY,204
AMD,1324
BROS,750
BYND,54


In [88]:
for k, v in accounts.items():
    print(k)
    check_symbols(v)
    print()

Rollover IRA
All clear

ROTH IRA
All clear

SIMPLE IRA
All clear



In [89]:
for v in accounts.values():
    out = pd.concat([out, v], axis=1)
out.fillna(0, inplace=True)
out = out.astype(int)
out.head()

,ET,Rollover IRA,ROTH IRA,SIMPLE IRA
AAPL,2875,0,605,0
ABMD,2264,1509,0,0
ABNB,196,0,489,0
ACN,286,0,0,0
ADBE,0,0,0,0


### TD Ameritrade

In [90]:
#!cat $DL/Positions_damiansp_Stocks_2022_08_09.xls

In [91]:
today = str(TODAY).replace('-', '_')
path = f'{DL}/Positions_damiansp_Stocks_{today}.xls'
#tomorrow = str(TODAY + timedelta(1)).replace('-', '_')
#path = f'{DL}/Positions_damiansp_Stocks_{tomorrow}.xls'

#path = f'{DL}/Positions_damiansp_Stocks_2022_11_04.xls'

In [92]:
# Stock (default) view
#td = pd.read_excel(, engine='xlrd')
def parse_file(path):
    inds = []
    data = []
    is_header = True
    with open(path, 'r') as f:
        for line in f:
            if not is_header:
                if line.startswith('"'):
                    try:
                        columns = line.split('\t')
                        symbol = columns[0][1:-1]
                        value = float(columns[5][1:-1].replace(',', ''))
                        inds.append(symbol)
                        data.append(value)
                    except IndexError:
                        print('Cannot parse:', line)
            if line.startswith('Symbol'):
                is_header = False
    return pd.DataFrame({'TD': data}, index=inds)

In [93]:
td = parse_file(path)
td.head()

,TD
AGI,133.80
AMEH,60.24
AMRK,223.72
APT,91.08
BROS,34.11


In [94]:
out = pd.concat([out, td], axis=1).fillna(0).round().astype(int)
out.head()

,ET,Rollover IRA,ROTH IRA,SIMPLE IRA,TD
AAPL,2875,0,605,0,0
ABMD,2264,1509,0,0,0
ABNB,196,0,489,0,0
ACN,286,0,0,0,0
ADBE,0,0,0,0,0


In [95]:
out.to_csv(f'{DL}/clean.csv')

In [96]:
!open $DL/clean.csv

In [31]:
!rm $DL/$filename $DL/Positions.csv $DL/clean.csv
!ls $DL

### MarketWatch

In [10]:
!mv "$DL/Holdings - Damian Satterthwaite-Phillips.csv" "$DL/Holdings - Damian Satterthwaite-Phillips(0).csv"
!ls $DL

Holdings - Damian Satterthwaite-Phillips(0).csv
Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv
Portfolio_Positions_Sep-02-2022.csv
Positions.csv
Positions_damiansp_Stocks_2022_09_02.xls


In [11]:
dfs = [binder]
for i, f in enumerate(sorted(os.listdir(DL))):
    print(f)
    if f.startswith('Holdings'):
        mw = pd.read_csv(f'{DL}/{f}')[['Symbol', 'Value']]
        mw.rename(columns={'Value': f'Value_{i}'}, inplace=True)
        mw.index = mw.Symbol
        mw.drop('Symbol', axis=1, inplace=True)
        mw[f'Value_{i}'] = mw[f'Value_{i}'].apply(convert_value)
        dfs.append(mw)
mw = pd.concat(dfs, axis=1).fillna(0).astype(int)
mw.head()

.DS_Store
Holdings - Damian Satterthwaite-Phillips(0).csv
Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv
Portfolio_Positions_Sep-02-2022.csv
Positions.csv
Positions_damiansp_Stocks_2022_09_02.xls


,Value_1,Value_2,Value_3
AAPL,779,2337,1558
ABMD,1045,784,0
ABNB,682,568,1136
ADBE,736,1104,0
ADYEY,602,514,1306


In [12]:
check_symbols(mw)

All clear


In [13]:
#out = pd.concat([binder, mw], axis=1).fillna(0).astype(int)
mw.head()

,Value_1,Value_2,Value_3
AAPL,779,2337,1558
ABMD,1045,784,0
ABNB,682,568,1136
ADBE,736,1104,0
ADYEY,602,514,1306


In [14]:
mw.to_csv(f'{DL}/mw.csv')

In [47]:
!open $DL/mw.csv

The file /Users/damiansp/Downloads/mw.csv does not exist.


In [ ]:
!rm $DL/Holdings*.csv $DL/mw.csv
!ls $DL

In [ ]:
!rm $DL/*